# Recurrent neural networks and their modifications

In [17]:
import os
import numpy as np

import zipfile

from sklearn.model_selection import train_test_split

# data path initialization
BASE_DIR = '../'
TEXT_DATA_DIR = BASE_DIR + 'data/'
TEXT_DATA_FILE = "movie_reviews.csv"
HEADER = True

# parameters initialization
VALIDATION_SPLIT = 0.1
RANDOM_SEED = 42


def load_data():
    # function for loading data
    x = []
    y = []
    with open(os.path.join(TEXT_DATA_DIR, TEXT_DATA_FILE), "r", encoding="utf-8") as f:
        if HEADER:
            _ = next(f)
        for line in f:
            temp_y, temp_x = line.rstrip("\n").split(",", 1)
            x.append(temp_x.replace("'", ""))
            y.append(temp_y)
    return x, y

data, labels = load_data()
labels = np.asarray(labels, dtype='int8')

# spliting our original data on train and validation sets
data_train, data_val, labels_train, labels_val = \
    train_test_split(data, np.asarray(labels, dtype='int8'),
                     test_size=VALIDATION_SPLIT, random_state=RANDOM_SEED, stratify=labels)

In this workshop we will use pre-trained vectors, since our text body is not large enough to train own embeddings. For the calculation speed, we will use [glove vectors of dimension 50](http://nlp.stanford.edu/data/glove.6B.zip).

Convert words to vectors.

In [34]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# инициализируем параметры словаря и эмбеддингов
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 40

print("Original sentence:\n", data_train[0])

# create a dictionary with Tokenizer
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='#$%&()*+-/:;<=>@[\\]^{|}~\t\n,.!"')
tokenizer.fit_on_texts(data_train)

# replacing words with their indexes from our dictionary
X_train = tokenizer.texts_to_sequences(data_train)
X_test = tokenizer.texts_to_sequences(data_test)

print("Sentence in indexes:\n", X_train[0])

# обрезаем каждое предложение приводим к нужной длинне
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

print("Sentence fitted to max length:\n", X_train[0])

Original sentence:
 "Filled with sentimentality, pretensions, unfulfilled ambitions and a host of dull characters faced with life threatening problems that verge on the ludicrous."
Sentence in indexes:
 [958, 13, 4032, 8229, 6794, 3, 2, 2910, 4, 676, 95, 2286, 13, 107, 4109, 716, 11, 7462, 19, 1, 2513]
Sentence fitted to max length:
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0  958   13 4032 8229 6794    3    2 2910    4  676   95
 2286   13  107 4109  716   11 7462   19    1 2513]


## Task 1
The `Tokenizer` object contains all the information about our dictionary. You should find the index of the word "nice" and how many times it was found in our train data.

In [35]:
# Replace 0 with a right answer
print("'nice' index – {}.".format(0))
print("The word 'nice' was found {} times.".format(0))

'nice' index – 0.
The word 'nice' was found 0 times.


<details>
  <summary>Here is a right answer</summary>
    <pre>
      <code>
  print("'nice' index – {}.".format(tokenizer.word_index['nice']))
  print("The word 'nice' was found {} times.".format(tokenizer.word_counts['nice']))
      </code>
    </pre>

</details>

In [36]:
# path to embeddings file
EMBEDDINGS_DIR = BASE_DIR + 'embeddings'
EMBEDDINGS_FILE = 'glove.6B.50d.txt'

EMBEDDING_DIM = 50

# choose only 10000 words from our dictionary
first_10000 = {k: v for k, v in tokenizer.word_index.items() if v < 10000}

# upload embeddings
embeddings = {}
with zipfile.ZipFile(os.path.join(EMBEDDINGS_DIR, EMBEDDINGS_FILE+'.zip')) as myzip:
    with myzip.open(EMBEDDINGS_FILE) as f:
        for line in f:
            values = line.split()
            word = values[0].decode('UTF-8')
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings[word] = coefs
        del values, word, coefs, line
print("Number of words with vector representation:", len(embeddings))

Number of words with vector representation: 400000


## Task 2
Find how many of the most common 10,000 words from our dictionary are not in the embedding dictionary. How can this amount be reduced?

In [ ]:
# Your solution

<details>
  <summary>Here is a right answer</summary>
      <pre>
            <code>
            len(set(first_10000.keys()).difference(embeddings.keys()))
            </code>
            Add more filters to the argument `filters` of `Tokenizer` object.
      </pre>

</details>

In [39]:
# prepare embeddings matrix where each row is word index

embedding_matrix = np.zeros((tokenizer.num_words, EMBEDDING_DIM))
for word, i in first_10000.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## Recurrent neural networks (RNN)

The simplest RNN formulation that is sensitive to the ordering of elements in the sequence is known as an Elman Network or Simple-RNN (S-RNN). The S-RNN was proposed by [Elman](http://onlinelibrary.wiley.com/doi/10.1207/s15516709cog1402_1/abstract;jsessionid=BC63955BD05165200EF5A1117E134554.f02t04) and explored for use in language modeling by [Mikolov](http://www.fit.vutbr.cz/~imikolov/rnnlm/google.pdf).

Recurrent neural networks help to grasp / understand the regularity, which depends on time or order. For example, when we try to classify an episode from a movie, it is important for us to know what was a couple of episodes earlier, or to understand the meaning of a certain word, we need to know the context that was before it.

A simple recurrent neural network has the following mathematical representation:<br><br>
$$\large h_t = \phi(Wx_t + Uh_{t-1})$$<br>
$$\large y = Vh_t$$

A formula illustration:
<img src="http://i.imgur.com/ifQrKRR.png" alt="rnn" style="width: 700px;"/>

It is easy to see that an unrolled RNN is just a very deep neural network (or rather, a very large computation graph with somewhat complex nodes), in which the same parameters
are shared across many parts of the computation, and additional input is added at various layers. To train an RNN network, then, all we need to do is to create the unrolled computation graph for a given input sequence, add a loss node to the unrolled graph, and then use the backward (backpropagation) algorithm to compute the gradients with respect to that loss. This procedure is referred to in the RNN literature as backpropagation through time ([BPTT](http://ieeexplore.ieee.org/document/58337/))

## Task 3

Draw an unrolled Simple Recurrent Network at time steps from k-2 to k.

<details>
  <summary>Here is a right answer</summary>
     <img src="https://photos-4.dropbox.com/t/2/AACkZO6XE2d62AQPmvsvtdX_kX5XxKtTbSFITuNHyzCZJA/12/533843084/png/32x32/1/_/1/2/unrolled_rnn.png/EKyZ0aIEGN0iIAIoAg/rJwBiWaoT5zQ4DS_oaXRKnJuGf7E6mJvCE1S3E-Iebg?size=2048x1536&size_mode=3" alt="unrolled_rnn" style="width: 700px;"/>

</details>

In [40]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Embedding
from keras.layers import SimpleRNN
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

NAME = "simple_rnn"

# embedding layer initialization

embedding_layer = Embedding(tokenizer.num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
                            
model = Sequential()
model.add(embedding_layer)
model.add(SimpleRNN(100))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# callbacks initialization
# automatic generation of learning curves
callback_1 = TensorBoard(log_dir='./logs/logs_{}'.format(NAME), histogram_freq=0,
                             write_graph=False, write_images=False)
# stop training model if accuracy does not increase more than five epochs
callback_2 = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto')
# best model saving
callback_3 = ModelCheckpoint("../models/model_{}.hdf5".format(NAME), monitor='val_acc',
                                 save_best_only=True, verbose=1)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()
#model.fit(X_train, labels_train, validation_data=[X_test, labels_test], 
#          batch_size=1024, epochs=100, callbacks=[callback_1, callback_2, callback_3])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 50)            500000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 100)               15100     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 515,201
Trainable params: 15,201
Non-trainable params: 500,000
_________________________________________________________________


If you train this model, then the accuracy on the training sample will be **77.13%**, and on validation - **75.60%**.

The S-RNN is hard to train effectively because of the vanishing gradients problem ([Pascanu et al., 2012](https://arxiv.org/abs/1211.5063)). Error signals (gradients) in later steps in the sequence diminish quickly in the backpropagation process, and do not reach earlier input signals, making it hard for the S-RNN to capture long-range dependencies. Gating-based architectures, such as the LSTM ([Hochreiter and Schmidhuber, 1997](http://www.bioinf.jku.at/publications/older/2604.pdf)) and the GRU ([Cho et al., 2014b](https://arxiv.org/abs/1406.1078)) are designed to solve this deficiency.

Consider the RNN as a general purpose computing device, where the state $s_i$ represents a finite memory. Each application of the function R reads in an input $x_{i+1}$, reads in the current memory $s_i$ , operates on them in some way, and writes the result into memory, resulting in a new memory state $s_{i+1}$. Viewed this way, an apparent problem with the S-RNN architecture is that the memory access is not controlled. At each step of the computation, the entire memory state is read, and the entire memory state is written.

***

## Long short-term memory (LSTM)

LSTM has a number of advantages over a simple recurrent neural network. LSTM is able to store the necessary information about a certain object and does not pay attention to not actual information. For example, a scene without a mention of the main character will not change information about him and it will focus with the mention.

- ** Adding a forgetting mechanism. ** If the episode ends, for example, the model should forget the current location, time of day and forget any information about the particular scene. However, if the character dies in the episode, the network must continue to remember that he is no longer alive. Thus, we want the model to study a separate forget / remember mechanism: when new input data appear, it must know what facts to keep or throw away.

- ** Adding a saving mechanism. ** When the model sees a new episode, it needs to decide whether it is worth using and storing any information about it. You saw some new meme, but who cares?

- Therefore, when a new entry comes in, the model first forgets the long-term information that it decides that it is no longer needed. Then it learns which part of the new data should be used and stores them in its long-term memory.

- ** Focusing long-term memory into working memory. ** Finally, the model should find out which parts of its long-term memory are now useful. For example, the age of the hero can be useful information to keep in the long term (children are more likely to crawl, adults are more likely to work), but probably does not matter if he is not in the current scene. So, instead of using full long-term memory all the time, he learns which parts to focus on.

The advantage of LSTM over RNN is that RNN can overwrite its memory at each time step in a fairly uncontrolled fashion, but LSTM is more flexible and can store long-term information, focusing on the right parts of it.

Now consider this all from the side of mathematics.

Let's start with ** long-term memory **. First, we need to know which fragments of long-term memory ** continue to remember and which to forget **, so we need to understand, based on the new input and our working memory, what part of the long-term memory should be keep.

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png" alt="forget" style="width: 600px;"/>

Thus, we get the value between 0 and 1 as an output, where 0 - we forget everything, 1 - we remember everything.

Now it is necessary to decide what new information to remember and what part of it to add to the long-term memory:

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png" alt="add" style="width: 600px;"/>

This step consists of two parts: the first is what kind of information we want to update ($ \large i_t $); And the second - the candidate addition to long-term memory ( $\large \tilde{C_t}$).

Now we need to update our long-term memory:

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png" alt="add" style="width: 600px;"/>

After we have updated our long-term memory, we need to focus on the right information for a specific example.

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png" alt="add" style="width: 600px;"/>

In simple words, if we focus on some information, then the activation of the sigmoid returns 1, and if some information is not needed now, then we return 0.

Let's look at a toy example for better understanding. We will train on Pokemons :)

What "thinks" a fully connected neural network:
<img src="http://i.imgur.com/cOGzJxk.png" alt="pokemon_nn" style="heigh: 100px;"/>

What "thinks" is a simple recurrent network:
<img src="http://i.imgur.com/PnWiSCf.png" alt="pokemon_rnn" style="heigh: 100px;"/>

As can be seen from the picture, the recurrent network remembers what happened a couple of seconds ago and can roughly understand what caused the appearance of water in the next episode.

What "thinks" LSTM:
<img src="http://i.imgur.com/EGZIUuc.png" alt="pokemon_lstm" style="heigh: 100px;"/>

LSTM recalls what happened in the previous episode, also recall long-term memory and focuses only on the right information for a specific episode.

Sources:
1. [Exploring LSTMs](http://blog.echen.me/2017/05/30/exploring-lstms/)
2. [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
3. [Neural Network Methods for Natural Language Processing by Yoav Goldberg](http://www.morganclaypool.com/doi/abs/10.2200/S00762ED1V01Y201703HLT037)

In [41]:
from keras.layers import LSTM

# инициализируем слой эмбеддингов
NAME = "simple_lstm"

embedding_layer = Embedding(tokenizer.num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
                            
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(100))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()
#model.fit(X_train, labels_train, validation_data=[X_test, labels_test], 
#          batch_size=1024, epochs=100, callbacks=[callback_1, callback_2, callback_3])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 50)            500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 560,501
Trainable params: 60,501
Non-trainable params: 500,000
_________________________________________________________________


Parameters in a simple LSTM are almost 4 times larger, the mathematical exposure to this fact can be seen from the formulas above.

When replacing a simple RNN with LSTM, the accuracy in the training sample increased to **82.49%**, and on validation - **77.71%**.

***

# Gated Recurrent Unit (GRU)

The LSTM architecture is very effective, but also quite complicated. The complexity of the system makes it hard to analyze, and also computationally expensive to work with. The gated recurrent unit (GRU) was introduced by Cho et al. as an alternative to the LSTM. It was
subsequently shown to perform comparably to the LSTM on several (non-textual) datasets. Like the LSTM, the GRU is also based on a gating mechanism, but with substantially fewer gates and without a separate memory component.

It combines the forget and input gates into a single “update gate.” It also merges the cell state and hidden state, and makes some other changes.

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-GRU.png" alt="pokemon_lstm" style="heigh: 100px;"/>

In [42]:
from keras.layers import GRU

# инициализируем слой эмбеддингов
NAME = "simple_lstm"

embedding_layer = Embedding(tokenizer.num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
                            
model = Sequential()
model.add(embedding_layer)
model.add(GRU(100))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()
#model.fit(X_train, labels_train, validation_data=[X_test, labels_test], 
#          batch_size=1024, epochs=100, callbacks=[callback_1, callback_2, callback_3])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 40, 50)            500000    
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               45300     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
_________________________________________________________________
activation_3 (Activation)    (None, 1)                 0         
Total params: 545,401
Trainable params: 45,401
Non-trainable params: 500,000
_________________________________________________________________


There are fewer number of parameters in a simple GRU than in LSTM, so our model should less tend to overfitting on small datasets.

When replacing LSTM with GRU, the accuracy in the training sample still almost the same – **82.38%** and increased on validation - **78.49%**.

## Task 4

Now let's try to improve our simple LSTM model by adding the following modifications (the number of parameters should not change):
<details>
  <summary>Here is a right answer!</summary>
      <pre>
          1. Dropouts (Reduce overfitting).
          2. Masking (This parameter should be added to the initialization of embedding, so that the loss function does not take into account 0, when our input is less than the maximum length).
          3. Regularization(This approach often works, but not in the case of LSTM. l1 / l2 regularization is often used to prevent the explosion of gradients, but the LSTM cell is constructed in such a way that there are no explosions, so the use of l1 / l2 regularization is impractical and worsens the results).
      </pre>

</details>

In [43]:
from keras.layers import Dropout

# инициализируем слой эмбеддингов
NAME = "modified_lstm"

embedding_layer = Embedding(tokenizer.num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False,
                            mask_zero=True)
                            
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(LSTM(100, dropout=0.1, recurrent_dropout=0.1))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()
#model.fit(X_train, labels_train, validation_data=[X_test, labels_test], 
#          batch_size=1024, epochs=100, callbacks=[callback_1, callback_2, callback_3])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 40, 50)            500000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total params: 560,501
Trainable params: 60,501
Non-trainable params: 500,000
_________________________________________________________________

This model showed itself, as we expected, better: on the training sample - **78.99%**, on the validation sample - **79.81%**.

***

# Bidirectional and stacked LSTM

We can create more complicated models using RNNs. One of them are bidirectional and stacked LSTMs. In simple LSTM we go only from left to right, but in bidirectional we create to independent LSTMs: in the first we go as usual from left to right, and in another one – from right to left. Then concat output sequence before fully connected layer.

<img src="http://doc.paddlepaddle.org/release_doc/0.9.0/doc/_images/bi_lstm1.jpg" alt="pokemon_lstm" style="heigh: 100px;"/>

In [44]:
from keras.layers import Bidirectional

# инициализируем слой эмбеддингов
NAME = "bidirectional_lstm"

embedding_layer = Embedding(tokenizer.num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False,
                            mask_zero=True)
                            
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(100, dropout=0.1, recurrent_dropout=0.1)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()
#model.fit(X_train, labels_train, validation_data=[X_test, labels_test], 
#          batch_size=1024, epochs=100, callbacks=[callback_1, callback_2, callback_3])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 40, 50)            500000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 40, 50)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               120800    
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 201       
_________________________________________________________________
activation_5 (Activation)    (None, 1)                 0         
Total params: 621,001
Trainable params: 121,001
Non-trainable params: 500,000
________________________________________________________________

We got twice as many parameters as expected. Train accuracy is **81.55%**, validation – **80.10%**. The accuracy is higher than for simple lstm. In big datasets it should work even more better.

***

Stacked LSTM is another good modification. In that way we want to learn higher-level temporal representations.

In [45]:
embedding_layer = Embedding(tokenizer.num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False,
                            mask_zero=True)

model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(LSTM(100, recurrent_dropout=0.1, dropout=0.1, return_sequences=True))
model.add(LSTM(100, recurrent_dropout=0.1, dropout=0.1))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()
#model.fit(X_train, labels_train, validation_data=[X_test, labels_test], 
#          batch_size=1024, epochs=100, callbacks=[callback_1, callback_2, callback_3])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 40, 50)            500000    
_________________________________________________________________
dropout_5 (Dropout)          (None, 40, 50)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 40, 100)           60400     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
_________________________________________________________________
activation_6 (Activation)    (None, 1)                 0         
Total para

Stacked LSTM performes a little bit worse than bidirectional LSTM: train – **79.09%**, validation – **79.80%**.

As you can see, models performance is very close, so it worth to try different architectures and regularization techniques to get the best accuracy on a specific task.

These approaches are basic in applying recurrent neural networks for the sentiment analysis problem. Improve accuracy can be the following modifications:

- size of embeddings;
- the number of words in the dictionary;
- the maximum long sentence;
- modification of the network architecture;
- ensembles.